In [ ]:
import json
paper_lists = json.loads(open('abstract_crawler_output.json', 'r').read())

In [ ]:
import pandas as pd
df = pd.DataFrame(paper_lists, columns=["year", "title", "doi", "authors", "ccf_rank", "abbreviation", "ccf_name", "full_name", "publisher", "abstract"])\
    .sort_values(['ccf_rank', 'year'], ascending=[True, False])
df

In [ ]:
# ------------- Paper Quality -------------
df2 = df[(df['year'] >= 2019) | (df["year"] == -1)]
df2 = df2[(df2['ccf_rank'].str.contains('CCF-A|CCF-B', case=False))]
# ------------- Field         -------------
df2 = df2[(df2['title'].str.contains('', case=False))]
df2 = df2[(df2['abstract'].str.contains('reinforcement|multi-armed', case=False))]
# ------------- Direction     -------------
df2 = df2[(df2['title'].str.contains('', case=False))]
df2 = df2[(df2['abstract'].str.contains('adversarial', case=False))]
# ------------- Conference / Journal   -------------
df2 = df2[(df2['ccf_name'].str.contains('', case=False))]

df2 = df2.sort_values(['ccf_rank', 'year', 'abbreviation'], ascending=[True, False, False])
df3 = df2[["ccf_rank", "ccf_name", "year", "title", "authors", "abstract"]].copy() # Show
df3.authors = df3['authors'].str[:32]
print('Count: ', len(df3))
display(df3.head(20).style)
display(pd.DataFrame(df2.groupby(['ccf_rank', 'ccf_name']).size().sort_values(ascending=False)).style)

In [ ]:
target_df = df2.drop(index=[])
output = '\n'
for cla in target_df['ccf_rank'].unique():
    class_item = target_df[target_df['ccf_rank'] == cla]
    output += '## %s (%d)\n' % (cla, len(class_item))
    for v in class_item.groupby('ccf_name').size().sort_values(ascending=False).index:
        item = class_item[class_item['ccf_name'] == v]
        output += '### %s (%d)\n\n' % (v, len(item))
        for i in range(len(item)):
            output += '%d. **[%s](%s)**\n\n' % (i + 1, item.iloc[i].title, item.iloc[i].doi)
            output += '    **Authors**: %s\n\n' % (item.iloc[i].authors)
            output += '    **Year**: %d\n\n' % (item.iloc[i].year)
            output += '    **Abstract**:\n\n'
            output += '    %s\n\n' % (item.iloc[i].abstract.replace('\n', '\n\n    '))
open('abstract_output.md', 'w').write(output)